In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch import optim 
import copy 
import numpy as np 
import math 



In [ ]:
import random 

def exponential_decrease(idx_decoder,p=3):
    return math.exp(-p*idx_decoder)
for i in range(3):
    print(exponential_decrease(i))

A= torch.arange(6,dtype=torch.float32).reshape(2,3)
B= torch.tensor([[1.0,2.0,3.0], [1.0,2.0,3.0]])
C= torch.rand(2,3,4)

softmx_dim_1= nn.Softmax(dim=-1)
softmx_dim1= nn.Softmax(dim=1)
softmx_0= nn.Softmax(dim=0)
print(A,C )
print(softmx_dim_1(A)) # add the valus of rows and devide the number of i,j element by sum(rows - i)
print(softmx_dim1(A))
print(f'{softmx_dim1(B)}, {softmx_0(B)},{softmx_dim_1(B)}')
print(f' dim = 1 {softmx_dim1(C)}, dim =0  {softmx_0(C)},dim = -1 {softmx_dim_1(C)}')

In [48]:
import numpy as np 
import torch

# Batch size and total sequence length
# batch_size = 2
# total_seq_length = 10
# # Actual lengths of each sequence in the batch
# actual_lengths = [8, 9]
# # Create an empty mask filled with zeros of shape [batch_size, total_seq_length]
# padding_mask = torch.zeros((batch_size, total_seq_length))
# # Set positions corresponding to the actual lengths to 1
# for i, length in enumerate(actual_lengths):
#     padding_mask[i, :length] = 1

# print(padding_mask.shape)

padding_mask = torch.zeros((2, 10))
padding_mask[0, :8] = 1  # First sequence has 8 actual values
padding_mask[1, :9] = 1 
padding_mask= padding_mask.unsqueeze(1)

proj_query= torch.rand(2,64,10)
proj_key= torch.rand(2,64,10)
proj_val=torch.rand(2,64,10)
m,c1,l1= proj_query.shape
m,c2,l2= proj_key.shape 
energy = torch.bmm(proj_query.permute(0,2,1),proj_key) # bmm stands for batch matrix multiplication 
attention = energy/np.sqrt(c1)
print(f' query {proj_query.shape} key {proj_key.shape} energy {energy.shape} c1 {c1} and  attention  {attention.shape} ')
attention = attention + torch.log(padding_mask+ 1e-6)
print(attention.shape, torch.log(padding_mask + 1e-6) )
softmax=nn.Softmax(dim=-1)
attention= softmax(attention)
attention= attention * padding_mask
print(attention.shape)
attention = attention.permute(0,2,1)
print(attention.shape)

out = torch.bmm(proj_val,attention)
print(out.shape)

bl=15
window_mask = torch.zeros((1, bl, bl + 2* (bl //2))) # 1,16,32 | 1,15,29
for i in range(bl):
    window_mask[:, :, i:i+bl] = 1
print(window_mask.shape, len(window_mask.shape))

 query torch.Size([2, 64, 10]) key torch.Size([2, 64, 10]) energy torch.Size([2, 10, 10]) c1 64 and  attention  torch.Size([2, 10, 10]) 
torch.Size([2, 10, 10]) tensor([[[ 9.5367e-07,  9.5367e-07,  9.5367e-07,  9.5367e-07,  9.5367e-07,
           9.5367e-07,  9.5367e-07,  9.5367e-07, -1.3816e+01, -1.3816e+01]],

        [[ 9.5367e-07,  9.5367e-07,  9.5367e-07,  9.5367e-07,  9.5367e-07,
           9.5367e-07,  9.5367e-07,  9.5367e-07,  9.5367e-07, -1.3816e+01]]])
torch.Size([2, 10, 10])
torch.Size([2, 10, 10])
torch.Size([2, 64, 10])
torch.Size([1, 15, 29]) 3


In [ ]:
import numpy as np 
class AttentionHelper(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.softmax=nn.Softmax(dim=-1)
    def scalar_dot_att(self,proj_query,proj_key, proj_val, padding_mask):
        '''A
        '''
        m,c1,l1= proj_query.shape
        m,c2,l2= proj_key.shape 

        assert c1==c2

        energy= torch.bmm(proj_query.permute(0,2,1),proj_key)
        attention = energy / np.sqrt(c1)
        if len(padding_mask.shape) != len(attention.shape):
            padding_mask = padding_mask.unsqueeze(1)

        attention = attention + torch.log(padding_mask + 1e-6) # mask the zero paddings. log(1e-6) for zero paddings
        attention = self.softmax(attention) 
        attention = attention * padding_mask
        attention = attention.permute(0,2,1)
        out = torch.bmm(proj_val, attention)
        return out, attention
    
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class AttLayer(nn.Module):
    def __init__(self,q_dim,k_dim, v_dim,r,block, stage,att_type): # r is the reduction factor 
        super(AttLayer, self).__init__()
        self.query_conv = nn.conv1D(in_channel= q_dim, out_channel= q_dim//r, kernel_size=1)
        self.key_conv = nn.conv1D(in_channel= k_dim, out_channel= k_dim//r, kernel_size=1)
        self.value_conv = nn.conv1D(in_channel= v_dim, out_channel= v_dim//r, kernel_size=1)

        conv_out= nn.conv1D(in_channel= v_dim//r, out_channels= v_dim, kernel_size=1)
        self.block = block
        self.stage= stage 
        self.att_type= att_type

        assert self.stage in ['encoder', 'decoder']
        assert self.att_type in ['normal_att', 'block_att','sliding_att']

        self.att_helper= AttentionHelper()
        self.window_mask= self.construct_window_mask()
    def forward(self, x1, x2, mask):
        # x1 from the encoder
        # x2 from the decoder
        
        query = self.query_conv(x1)
        key = self.key_conv(x1)
         
        if self.stage == 'decoder':
            assert x2 is not None
            value = self.value_conv(x2)
        else:
            value = self.value_conv(x1)
            
        if self.att_type == 'normal_att':
            return self._normal_self_att(query, key, value, mask)
        elif self.att_type == 'block_att':
            return self._block_wise_self_att(query, key, value, mask)
        elif self.att_type == 'sliding_att':
            return self._sliding_window_self_att(query, key, value, mask)


    def construct_window_mask(self):
        window_mask= torch.zeros((1,self.block,self.block+ 2*(self.block//2)))
        for i in range(self.block):
            window_mask[:,:,i:i+self.block]= 1
        return window_mask.to(device)
    
    def _sliding_window_self_att(self,q,k,v,mask):
        m_batchsize,c1,L = q.size()
        _, c2, _ = k.size()
        _,c3,_ = v.size()

        nb= L//self.block

        if L% self.block !=0:  # there is reminder - the remaining sequences 
            q= torch.cat([q,torch.zeros((m_batchsize,c1,self.block- L % self.block)).to(device)], dim=-1)
            k= torch.cat([k,torch.zeros((m_batchsize,c2,self.block - L//self.block)).to(device)], dim =-1)
            v= torch.cat([v, torch.zeros((m_batchsize,self.block- L//self.block)).to(device)], dim = -1 )

        padding_mask = torch.cat([torch.ones((m_batchsize,1,L)).to(device) * mask[:,0:1,:], torch.zeros((m_batchsize,1,self.block * nb- L)).to(device)], dim = -1 )

        # lets expand the block to cover the 1/2 of prior window and posterior window 
        # the querry is the sliding window and we need to expand the key and value vector 

        q= q.reshape(m_batchsize,c1,nb,self.block).permute(0,2,1,3).reshape(m_batchsize*nb , c1,self.block)
        # extend the k and v for 
        k = k.reshape(m_batchsize,c2,)

        


        


In [ ]:
from libs.utils.preprocess import load_apply_sam_model
